## This kaggle notebook extensively uses code from Kostiantyn Isaienkov's notebook: 
## Keras autoencoder (DAE) + neural network starter

## Link: https://www.kaggle.com/isaienkov/keras-autoencoder-dae-neural-network-starter

## We wish to test our own custom NN using Isaienkov's notebook's preprocessing steps 

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/lish-moa/train_targets_scored.csv
/kaggle/input/lish-moa/sample_submission.csv
/kaggle/input/lish-moa/train_drug.csv
/kaggle/input/lish-moa/train_targets_nonscored.csv
/kaggle/input/lish-moa/train_features.csv
/kaggle/input/lish-moa/test_features.csv


In [2]:
import numpy as np
import pandas as pd


In [3]:
import tensorflow as tf

from keras.models import Sequential
from keras.layers import Dense

from sklearn.metrics import log_loss

In [4]:
train_features = pd.read_csv('/kaggle/input/lish-moa/train_features.csv')
train_targets = pd.read_csv('/kaggle/input/lish-moa/train_targets_scored.csv')
test_features = pd.read_csv('/kaggle/input/lish-moa/test_features.csv')
submission = pd.read_csv('/kaggle/input/lish-moa/sample_submission.csv')

In [5]:
train_targets

,sig_id,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,...,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
0,id_000644bb2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,id_000779bfc,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,id_000a6266a,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,id_0015fd391,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,id_001626bd3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23809,id_fffb1ceed,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
23810,id_fffb70c0c,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
23811,id_fffc1c3f4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
23812,id_fffcb9e7c,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
def preprocess(df):
    df.loc[:, 'cp_dose'] = df.loc[:, 'cp_dose'].map({'D1': 0, 'D2': 1})
    df.loc[:, 'cp_time'] = df.loc[:, 'cp_time'].map({24: 0, 48: 1, 72:2})
    del df['sig_id']
    return df

train = preprocess(train_features)
test = preprocess(test_features)

del train_targets['sig_id']

# train_features['cp_dose'] = train_features['cp_dose'].map({'D1': -1, 'D2': -2})
# train_features['cp_time'] = train_features['cp_time'].map({24: -1, 48: 0, 72:1})
# train_features = train_features[:,train_features.columns != 'sig_id'] 
# train_targets = train_targets[:,train_targets.columns  != 'sig_id'] 

train_targets['cp_type'] = train['cp_type']

train = train[train['cp_type'] != 'ctl_vehicle']
train_features = train_features[train_features['cp_type'] != 'ctl_vehicle']
#test_features = test_features[test_features['cp_type'] != 'ctl_vehicle']
train_targets = train_targets[train_targets['cp_type'] != 'ctl_vehicle']

train = train.drop(['cp_type'], axis=1)
train_features = train_features.drop(['cp_type'], axis=1)
test_features = test_features.drop(['cp_type'], axis=1)
train_targets = train_targets.drop(['cp_type'], axis=1)

train = train.reset_index().drop(['index'], axis=1)
train_features = train_features.reset_index().drop(['index'], axis=1)
test_features = test_features.reset_index().drop(['index'], axis=1)
train_targets = train_targets.reset_index().drop(['index'], axis=1)

In [7]:
train_categories = train[['cp_dose', 'cp_time']]
test_categories = test[['cp_dose', 'cp_time']]

test_cp_type = test['cp_type']
test = test.drop(['cp_type'], axis=1)

In [8]:
# model = Sequential([
#     Dense(320, activation='relu'),
#     Dense(32, activation='relu'),
#     Dense(320, activation='relu'),
#     Dense(32, activation='relu'),
#     Dense(206 , activation='sigmoid')
# ])

In [9]:
# model.compile(optimizer='adam',
#               loss='binary_crossentropy')

In [10]:
#NJ_hist = model.fit(train, train_targets, batch_size=1000, epochs=100)


In [11]:
    submission.loc[:, train_targets.columns] = 0
    model = Sequential([
    Dense(320, activation='relu'),
    Dense(32, activation='relu'),
    Dense(320, activation='relu'),
    Dense(32, activation='relu'),
    Dense(206 , activation='sigmoid')
    ])
    model.compile(optimizer='adam',
              loss='binary_crossentropy')
    
    NJ_hist = model.fit(train.values, train_targets.values, batch_size=1000, epochs=100)
    
    
    submission.loc[:, train_targets.columns] += model.predict(test_features)

Epoch 1/100
22/22 [==============================] - 0s 21ms/step - loss: 0.5265
Epoch 2/100
22/22 [==============================] - 0s 20ms/step - loss: 0.0937
Epoch 3/100
22/22 [==============================] - 0s 20ms/step - loss: 0.0277
Epoch 4/100
22/22 [==============================] - 0s 20ms/step - loss: 0.0231
Epoch 5/100
22/22 [==============================] - 0s 20ms/step - loss: 0.0217
Epoch 6/100
22/22 [==============================] - 0s 20ms/step - loss: 0.0210
Epoch 7/100
22/22 [==============================] - 0s 20ms/step - loss: 0.0204
Epoch 8/100
22/22 [==============================] - 0s 20ms/step - loss: 0.0199
Epoch 9/100
22/22 [==============================] - 0s 20ms/step - loss: 0.0195
Epoch 10/100
22/22 [==============================] - 0s 20ms/step - loss: 0.0192
Epoch 11/100
22/22 [==============================] - 0s 20ms/step - loss: 0.0188
Epoch 12/100
22/22 [==============================] - 0s 20ms/step - loss: 0.0185
Epoch 13/100
22/22 [=====

In [12]:
# submission.loc[:, train_targets.columns] = 0
# res = train_targets.copy()
# for n, (tr, te) in enumerate(KFold(n_splits=7, random_state=666, shuffle=True).split(train_targets)):
#     print(f'Fold {n}')
    
#     model = Sequential([
#     Dense(320, activation='relu'),
#     Dense(32, activation='relu'),
#     Dense(320, activation='relu'),
#     Dense(32, activation='relu'),
#     Dense(206 , activation='sigmoid')
#     ])
#     model.compile(optimizer='adam',
#               loss='binary_crossentropy')
    
#     NJ_hist = model.fit(train.values[tr], train_targets.values[tr], batch_size=1000, epochs=100)
    
    
#     submission.loc[:, train_targets.columns] += model.predict(test_features)
#     res.loc[te, train_targets.columns] = model.predict(train_features.values[te])
    
# submission.loc[:, train_targets.columns] /= (n+1)


In [13]:
# submission.loc[:, train_targets.columns] = 0
# res = train_targets.copy()
# for (tr, te) in train_targets:
#     print(f'Fold {n}')
    
#     model = Sequential([
#     Dense(320, activation='relu'),
#     Dense(32, activation='relu'),
#     Dense(320, activation='relu'),
#     Dense(32, activation='relu'),
#     Dense(206 , activation='sigmoid')
#     ])
#     model.compile(optimizer='adam',
#               loss='binary_crossentropy')
    
#     NJ_hist = model.fit(train.values[tr], train_targets.values[tr], batch_size=1000, epochs=100)
    
#     submission.loc[:, train_targets.columns] += model.predict(test_features)
#     res.loc[te, train_targets.columns] = model.predict(train_features.values[te])
    


In [14]:
res = train_targets.copy()
metrics = []
for _target in train_targets.columns:
    metrics.append(log_loss(train_targets.loc[:, _target], res.loc[:, _target]))

In [15]:
#submission.loc[:, train_targets.columns] += model.predict(test_features)

In [16]:
print(f'OOF Metric: {np.mean(metrics)}')

OOF Metric: 9.99200722162641e-16


In [17]:
submission['cp_type'] = test_cp_type
for col in submission.columns:
    if col in ['sig_id', 'cp_type', 'cp_dose', 'cp_time']:
        continue
    submission.loc[submission['cp_type'] == 'ctl_vehicle', col] = 0

submission = submission.drop(['cp_type'], axis=1)

In [18]:
submission.to_csv('submission.csv', index=False)

In [19]:
submission

,sig_id,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,...,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
0,id_0004d9e33,2.382296e-12,2.367399e-11,6.013774e-22,2.491950e-18,6.215796e-10,1.364753e-07,4.574143e-13,1.335580e-19,4.699510e-16,...,3.704619e-18,6.489062e-10,4.350616e-16,3.386829e-12,2.813937e-22,9.626667e-09,7.764444e-19,6.773319e-14,1.384563e-14,5.261823e-21
1,id_001897cda,2.006071e-16,5.100238e-16,8.941960e-21,1.532168e-33,1.580850e-37,6.651742e-22,2.647878e-23,1.030865e-17,6.214878e-23,...,2.634959e-09,1.178562e-16,1.377337e-05,1.268726e-10,4.751052e-09,6.942448e-19,7.225193e-14,5.612355e-14,7.812690e-20,7.238097e-16
2,id_002429b5b,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
3,id_00276f245,1.870514e-23,6.500623e-24,6.294309e-14,1.566297e-32,5.986306e-15,5.413105e-17,2.215449e-13,4.740897e-20,1.514602e-17,...,1.386037e-18,8.599458e-15,1.060271e-16,2.161546e-17,7.868235e-19,1.700638e-20,4.017450e-24,8.731149e-15,1.536757e-18,1.444464e-28
4,id_0027f1083,1.773670e-02,1.065668e-09,1.092084e-10,2.702108e-11,6.989866e-16,1.776001e-10,5.914700e-12,1.004972e-15,6.205966e-17,...,4.301035e-13,5.530455e-11,1.264475e-12,3.935961e-12,9.028099e-12,3.801985e-12,1.119079e-13,2.128168e-06,3.803282e-09,1.527982e-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3977,id_ff7004b87,9.995810e-15,4.498504e-25,1.276950e-11,0.000000e+00,2.336413e-32,1.841248e-22,2.737587e-29,1.506923e-20,3.244960e-19,...,3.754921e-14,3.116735e-18,1.038629e-18,6.130977e-16,9.199336e-08,3.725473e-14,1.255355e-09,6.185078e-08,1.435786e-09,1.490574e-13
3978,id_ff925dd0d,5.571546e-11,9.118447e-17,4.779403e-08,5.790980e-16,4.477395e-06,4.597497e-09,1.699293e-15,2.636116e-16,2.402800e-11,...,5.643194e-14,4.016975e-16,4.773398e-20,2.354261e-17,5.890139e-18,2.716623e-08,3.617164e-15,1.361922e-11,1.920103e-06,2.065728e-12
3979,id_ffb710450,8.379557e-12,7.320229e-17,3.372062e-12,4.763977e-19,1.266744e-09,9.711834e-13,1.354287e-09,7.833992e-14,1.007573e-17,...,2.015307e-15,4.852989e-13,9.037002e-13,1.908570e-16,8.629484e-17,1.045499e-13,4.416425e-19,1.790544e-12,1.345573e-14,1.391240e-17
3980,id_ffbb869f2,3.452826e-15,6.465832e-05,3.342481e-18,9.822210e-01,1.646286e-11,9.912795e-01,1.232305e-05,6.167975e-22,5.106466e-17,...,7.417411e-15,2.513229e-18,1.405674e-19,2.652295e-12,2.508928e-33,7.386799e-16,6.408917e-20,9.792972e-13,6.900061e-07,5.838623e-17
